In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm

In [ ]:
def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
#     tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(548, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.8),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(248, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                  loss='binary_crossentropy', 
                  )
    return model

In [ ]:
def custom_log_loss(y_true, y_pred):
    s = 0.0
    for i in range(y_true.shape[0]):
        s+=y_true[i]*np.log(y_pred[i])+(1-y_true[i])*np.log(1-y_pred[i])
    
    return s/(-y_true.shape[0])

In [ ]:
eps, weight_tg, weight_x3, weight_x5 = 10**(-5), 0.0, 0.0, 0.0 # last couple =0.0
weight = 1.0 - weight_tg - weight_x3 - weight_x5


def cube(x):
    x /= 10.0
    return 10.0 * x*x*x
def tgfunc(x):
    return  np.tan(np.pi *x/ 4.0) 
def application_transform(x):
    """    x = 2*x-1.0
    x1 = weight_tg*tgfunc(x) + (weight_x3 + weight_x5*x**2)*x**3 + weight*x
    x1 = x1/2 + 0.5"""
    x1 = x
    if x1 < eps:
        x1 = eps
    elif x1 >= 1 - eps:
        x1 = 1 - eps
    return x1

def transform(x):
    x = x[x['cp_type']=='trt_cp']
    x.pop('cp_type')
    x['cp_dose'] = x['cp_dose'].replace({'D1':-1, 'D2':1})
    x['cp_time'] = x['cp_time'] // 24
    x.join(pd.get_dummies(x['cp_time']))
    x.pop('cp_time')
    return x

X_all = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
y_all = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
X = transform(X_all[X_all.columns[1:]])
Y = y_all[X_all['cp_type']=='trt_cp']
Y = Y[Y.columns[1:]]
def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [ ]:
Y.transpose().iloc[1].transpose()

In [ ]:
def write_results(drug_no, val_loss, time_to_train):
    return {
        'drug_no': drug_no,
        #'random_state': random_state,
        'val_loss': val_loss,
        'time_to_train' : time_to_train
    }

In [ ]:
train_results = []
import time

In [ ]:

for drug_i in range(206):
#for random_state in range(2020,2070,10):
    Y_new = Y.transpose().iloc[drug_i].transpose()
    checkpoint_path = f'drug_no{drug_i+1}_rs.hdf5'
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=8)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')
    cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                 save_weights_only = True, mode = 'min')
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y_new, test_size=0.2 
                                                            #,random_state=random_state
                                                       )

    print(f'beginning training on drug no.{drug_i+1} ')
    model = create_model(X.shape[1])
    t0 = time.clock()
    model.fit(X_train, Y_train,
              validation_data=(X_test, Y_test), 
              epochs=50, batch_size=128,verbose=2,
              callbacks=[reduce_lr_loss, cb_checkpt, early_stopping])
    model.load_weights(checkpoint_path)
    t1 = time.clock()
    test_predict = np.array(list(map(application_transform, model.predict(X_test))))
    

    """df = pd.DataFrame(subm)
    df = df.applymap(application_transform)
    df = np.array(df).reshape(dim)
    """

    cur_loss = custom_log_loss(np.array(Y_test),test_predict)
    train_results.append(write_results(drug_i, cur_loss, t1-t0))
    print('==================================================')
    print(f'loss on validation: {cur_loss}')
    print(f'time for training: {t1-t0} sec')
    print('==================================================')

In [ ]:
pd.DataFrame(train_results).to_csv('train_res.csv',index=False)

In [ ]:
custom_log_loss(np.zeros(100), np.array(list(map(application_transform, np.zeros(100)))))